In [1]:
import os
os.chdir("../")

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [3]:
from RoadClassifier.constants import *
from RoadClassifier.utils.utils import read_yaml, create_directory
import tensorflow as tf

In [10]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directory([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directory([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "RoadImages")
        create_directory([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [11]:
import time

class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )

    def get_tb_ckpt_callbacks(self):
        return [
            self.create_tb_callbacks,
            self.create_ckpt_callbacks
        ]

In [12]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [13]:
import os
print(os.getcwd())

c:\Personal_Projects\All_Project\clean-littered-road-classification


In [14]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

[2025-01-01 19:24:48,641 : INFO utils] : yaml file: configs\config.yaml loaded successfully
[2025-01-01 19:24:48,677 : INFO utils] : yaml file: params.yaml loaded successfully
[2025-01-01 19:24:48,683 : INFO utils] : created directory at: artifacts
[2025-01-01 19:24:48,687 : INFO utils] : created directory at: artifacts\prepare_callbacks\checkpoint_dir
[2025-01-01 19:24:48,693 : INFO utils] : created directory at: artifacts\prepare_callbacks\tensorboard_log_dir
[2025-01-01 19:24:48,700 : INFO utils] : created directory at: artifacts\training
Found 46 images belonging to 2 classes.
Found 191 images belonging to 2 classes.


c:\Personal_Projects\All_Project\clean-littered-road-classification\env\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4550 - loss: 6.4744

c:\Personal_Projects\All_Project\clean-littered-road-classification\env\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


11/11 ━━━━━━━━━━━━━━━━━━━━ 50s 3s/step - accuracy: 0.4543 - loss: 6.3855 - val_accuracy: 0.6875 - val_loss: 1.4050
Epoch 2/10
 1/11 ━━━━━━━━━━━━━━━━━━━━ 5s 508ms/step - accuracy: 0.4375 - loss: 2.7445

c:\Personal_Projects\All_Project\clean-littered-road-classification\env\lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 254ms/step - accuracy: 0.4375 - loss: 2.7445 - val_accuracy: 0.6875 - val_loss: 0.9672
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.4842 - loss: 1.6508 - val_accuracy: 0.6875 - val_loss: 0.9462
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 249ms/step - accuracy: 0.4375 - loss: 1.5926 - val_accuracy: 0.6875 - val_loss: 0.6357
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.6260 - loss: 0.9051 - val_accuracy: 0.6875 - val_loss: 0.6390
Epoch 6/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 128ms/step - accuracy: 0.6250 - loss: 0.8483 - val_accuracy: 0.6875 - val_loss: 1.0541
Epoch 7/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.4624 - loss: 1.4886 - val_accuracy: 0.6875 - val_loss: 0.6584
Epoch 8/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 250ms/step - accuracy: 0.5000 - loss: 0.9836 - val_accuracy: 0.6875 - val_loss: 0.5975
Epoch 9/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.5192 - loss: 0.9051 - val_accuracy: 0.3125 - val_loss: 0.

In [15]:
model = tf.keras.models.load_model("./artifacts/training/model.keras")

In [16]:
from PIL import Image

img = Image.open("./artifacts/data_ingestion/RoadImages/dirty/dirty_4.jpg")
img = img.resize((224,224))

In [17]:
import numpy as np
img_array = np.array(img)

In [18]:
img_array.shape

(224, 224, 3)

In [19]:
img_array = np.expand_dims(img_array, axis=0)
img_array.shape

(1, 224, 224, 3)

In [20]:
result = model.predict(img_array)
result

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step


array([[0.69070643, 0.30929363]], dtype=float32)

In [22]:
argmax_index = np.argmax(result, axis=1)
if argmax_index[0] == 0:
    print("predicted: dirty road")
else:
    print("predicted: clean road")

predicted: dirty road
